In [ ]:
# Importação de bibliotecas
import tensorflow as tf
from pathlib import Path
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.layers import Dropout

In [ ]:
# Diretórios para dados de treinamento, validação e teste
path = Path("Expressao facial")
train_data_dir = path/'train'
validation_data_dir = path/'val'
test_data_dir = path/'test'

In [ ]:
# Dimensões das imagens e número de classes
img_width, img_height = 322, 322
num_classes = 3

In [ ]:
# Hiperparâmetros
batch_size = 16 # Numero de imagens por lote de treinamento
epochs = 15

In [ ]:
# Rescale = redimensiona os valores dos pixels para o intervalo [0, 1]
# Instancias do ImageDataGenerator são criadas para pré-processamento dos dados
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
# Carregamento e pré-processamento dos dados de treinamento
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb'  # Defina para 'rgb' para imagens coloridas, apesar da imagem estar em escala de cinza, a inceptionV3 espera uma imagem colorida
)

In [ ]:
# Carregamento e pré-processamento dos dados de validação
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb'
)


In [ ]:
# Carregamento e pré-processamento dos dados de teste
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=False,
    class_mode='categorical',
    color_mode='rgb'
)

In [ ]:
# Carregar InceptionV3 pré-treinado
base_model = InceptionV3(weights='imagenet', include_top=False)

In [ ]:
# Adicionar camadas personalizadas no topo do InceptionV3
model = base_model.output
model = GlobalAveragePooling2D()(model)
model = Dense(128,activation='relu')(model)
model = Dropout(0.15)(model)
model = Dense(256,activation='relu')(model)
model = Dropout(0.15)(model)
model = Dense(524,activation='relu')(model)
model = Dropout(0.15)(model)
model = Dense(num_classes, activation='softmax')(model)
model = Model(inputs=base_model.input, outputs=model)

model.summary()

In [ ]:
# Compilar o modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Treinamento do modelo com validacao
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

In [ ]:
# Dicionario que mapeia as classes
class_indices = {
    'happy': 0,
    'sad': 1,
    'surprise': 2
}

In [ ]:
# Associa o dicionario ao gerador de teste
test_generator.class_indices = class_indices

In [ ]:
# Mapeia indices de classe para nomes de classe
class_names = {v: k for k, v in test_generator.class_indices.items()}

In [ ]:
# Avaliacao do modelo no conjunto de teste
y_true = test_generator.classes
y_pred = model.predict(test_generator)
y_pred_classes = [tf.argmax(x).numpy() for x in y_pred]

In [ ]:
# Calcula metricas
accuracy = accuracy_score(y_true, y_pred_classes)
confusion = confusion_matrix(y_true, y_pred_classes)
classification_rep = classification_report(y_true, y_pred_classes, target_names=class_names.values(), digits=4)

print(f'Test Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{confusion}')
print(f'Classification Report:\n{classification_rep}')

In [ ]:
# Salvando o modelo treinado
model.save('inceptionrostos.h5')